In [2]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import random
import requests
import json

#Installing citipy
from citipy import citipy

#API key
from api_key import api_key
print(api_key)

4913eafb294e90c158bf2eb743930fd0


In [68]:
#DataFrame for city information
cities_df = pd.DataFrame(columns = ['City', 'Country', 'Lat', 'Lng'])

#Empty lists for iteration
city = []
country = []
t_lat = []
t_lng = []

#Generate random numbers for coordinates, then pass through citipy 
for i in range(20):
    rn_lat = round(random.uniform(-90, 90), 4)
    rn_lng = round(random.uniform(-180, 180), 4)
    
    coord = citipy.nearest_city(rn_lat, rn_lng)

    city.append(coord.city_name)
    country.append(coord.country_code)
    t_lat.append(rn_lat)
    t_lng.append(rn_lng)
    
#print(city)
#print(country)

In [42]:
#Add lists to dataframe
cities_df.City = city
cities_df.Country = country
#cities_df.head()

In [81]:
#Take citipy cities and pass through API
url_geo = 'http://api.openweathermap.org/geo/1.0/direct?q='

lat = []
lng = []
    